# 0. Google Colab Setups

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('./drive/MyDrive/NLP MedLM project')

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=74989bb86ffef5bdaa02dad5bdbd125ed6f7f605391eab16740c436f59b291df
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 1.5 MB/s eta 0:00:00


# 1. Load and Preprocess
+ Load all three test datasets: MedQuAD, iCliniq, and mixture of MedQuAD and iCliniq

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import torch
from transformers import BloomTokenizerFast, BloomForCausalLM, pipeline
from datasets import DatasetDict, Dataset
from transformers import Trainer, TrainingArguments
import evaluate

from tqdm import tqdm
import ast

In [ ]:
# load MedQuAD train and test dataset
i_test_df = pd.read_csv('./dataset/iCliniq_test.csv')
m_test_df = pd.read_csv('./dataset/MedQuAD_test.csv')
c_test_df = pd.read_csv('./dataset/combined_test.csv')

In [ ]:
# add special prompting tokens
def prepend_qa(row):
  return '[question] ' + str(row) + '\n [answer]'

i_test_df['Question'] = i_test_df['Question'].apply(prepend_qa)
m_test_df['Question'] = m_test_df['Question'].apply(prepend_qa)
c_test_df['Question'] = c_test_df['Question'].apply(prepend_qa)

In [ ]:
# check data
c_test_df.head(1)

,Question,Answer,Answer_cut
0,[question] can rupturing a cyst under the skin...,this would be highly unlikely however i do not...,this would be highly unlikely however i do not...


# 2. Inference
+ This notebook aims to use the un-finetuned model for question answering in order to measure the effectiveness of the finetuning.

In [ ]:
MODEL_TYPE = "bigscience/bloom-560m"
MAX_LENGTH = 300

tokenizer = BloomTokenizerFast.from_pretrained(MODEL_TYPE)
model = BloomForCausalLM.from_pretrained(MODEL_TYPE)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

cuda:0


## 2.1 Inference on MedQuAD

In [ ]:
# test questions as list
m_test_data = m_test_df['Question'].tolist()

In [ ]:
def split_data(dataset, chunk_size):
  print(len(dataset))
  sublists = [dataset[i:i+32] for i in range(0, len(dataset), 32)]
  print(len(sublists))
  return sublists

In [ ]:
m_sublists = split_data(m_test_data, 32)

3279
103


In [ ]:
# inference on all data
# test different batchsize to find the best suitable one
generator = pipeline('text-generation', model=model.to(device), device = 0, tokenizer=tokenizer, do_sample=False)
def inference(sublists):
  gen_ans_lists = []
  for sub_test_data in tqdm(sublists):
    with torch.no_grad():
      gen_ans = generator(sub_test_data, max_new_tokens=300, batch_size=32 )
      gen_ans_lists.append(gen_ans)
  return gen_ans_lists


In [ ]:

m_gen_ans_lists = inference(m_sublists)

 10%|▉         | 10/103 [01:39<15:18,  9.88s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 103/103 [16:48<00:00,  9.79s/it]


In [ ]:
# store the temporary generated answers
len(m_gen_ans_lists)
m_gen_ans_df = pd.DataFrame(m_gen_ans_lists)
m_gen_ans_df.to_csv("medQuAD_nofinetune_gen_ans.csv", index = False)

## 2.2 Inference on iCliniq

In [ ]:
i_test_data = i_test_df['Question'].tolist()

In [ ]:
i_sublists = split_data(i_test_data, 32)

5951
186


In [ ]:
i_gen_ans_lists = inference(i_sublists)

  5%|▌         | 10/186 [00:39<11:00,  3.75s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 186/186 [11:08<00:00,  3.60s/it]


In [ ]:
len(i_gen_ans_lists)
i_gen_ans_df = pd.DataFrame(i_gen_ans_lists)
i_gen_ans_df.to_csv("iCliniq_nofinetune_gen_ans.csv", index = False)

## 2.3 Inference on combined iCliniq and MedQuAD

In [ ]:
c_test_data = c_test_df['Question'].tolist()

In [ ]:
c_sublists = split_data(c_test_data, 32)

9229
289


In [ ]:
c_gen_ans_lists = inference(c_sublists)

  3%|▎         | 10/289 [01:06<30:04,  6.47s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 289/289 [31:23<00:00,  6.52s/it]


In [ ]:
len(c_gen_ans_lists)
c_gen_ans_df = pd.DataFrame(c_gen_ans_lists)
c_gen_ans_df.to_csv("combined_nofinetune_gen_ans.csv", index = False)

# 3. Evaluation
+ Use SacreBleu score and rouge for  evaluation

In [ ]:
# load the generated answers

def eval(datasetname, test_df):
  gen_ans_df = pd.read_csv("{}_nofinetune_gen_ans.csv".format(datasetname))
  # extract the generated answers with sequence
  gen_ans_list = []

  for index, row in gen_ans_df.iterrows():
      for col in gen_ans_df.columns:
        try:
          ans = ast.literal_eval(row[col])[0]['generated_text']
          # only keep the answer part
          prompt = "\n [answer]"
          prompt_index = ans.find(prompt)
          if prompt_index != -1:
            extracted_ans = ans[prompt_index + len(prompt): ]
          else: # handle edge case
            extracted_ans = ans
        except:
          continue
        gen_ans_list.append(extracted_ans)

  # check the number of datapoints
  print(len(test_df))
  print(len(gen_ans_list))

  # check a random sample's question and answer
  print("question: ", test_df['Question'].iloc[48])
  print("generated_answer: ", gen_ans_list[48])
  print("reference_answer: ", test_df['Answer'].iloc[48])

  #
  sacrebleu = evaluate.load("sacrebleu")
  references = [[ref_ans] for ref_ans in test_df['Answer']]
  predictions = gen_ans_list
  sacrebleu_results = sacrebleu.compute(predictions=predictions, references=references)
  print("sacrebleu: ", sacrebleu_results)

  rouge = evaluate.load("rouge")
  rouge_results = rouge.compute(predictions=predictions, references=references)
  print("rouge: ", rouge_results)

In [ ]:
# evaluate medquAd
eval('medQuAD', m_test_df)

3279
3279
question:  [question] What is (are) familial porencephaly ?
 [answer]
generated_answer:   The family is the most important factor in the development of the disease. The family is the most important factor in the development of the disease. The family is the most important factor in the development of the disease. The family is the most important factor in the development of the disease. The family is the most important factor in the development of the disease. The family is the most important factor in the development of the disease. The family is the most important factor in the development of the disease. The family is the most important factor in the development of the disease. The family is the most important factor in the development of the disease. The family is the most important factor in the development of the disease. The family is the most important factor in the development of the disease. The family is the most important factor in the development of the disease. 

sacrebleu:  {'score': 1.6760597710297152, 'counts': [126507, 20814, 5235, 2205], 'totals': [792715, 789436, 786157, 782878], 'precisions': [15.9586989018752, 2.6365658520766724, 0.6658975242858615, 0.28165308004567763], 'bp': 1.0, 'sys_len': 792715, 'ref_len': 735820}


rouge:  {'rouge1': 0.1387951873624898, 'rouge2': 0.02460650969486563, 'rougeL': 0.11497936858976178, 'rougeLsum': 0.11331596318592912}


In [ ]:
eval('iCliniq', i_test_df)

5951
5951
question:  [question] my mother gets head spins vision gets black or blurred?
 [answer]
generated_answer:   yes, it is possible. I have a mother who has a head spindle vision. I have a mother who has a head spindle vision. I have a mother who has a head spindle vision. I have a mother who has a head spindle vision. I have a mother who has a head spindle vision. I have a mother who has a head spindle vision. I have a mother who has a head spindle vision. I have a mother who has a head spindle vision. I have a mother who has a head spindle vision. I have a mother who has a head spindle vision. I have a mother who has a head spindle vision. I have a mother who has a head spindle vision
reference_answer:  that could be either due to a neck problem like cervical spondylosis or disc causing vertebro basilar insufficiency or due to bppv benign positional paroxysmal vertigo. get a duplex usg on neck arteries. cervical mri. and see a neurologist.


sacrebleu:  {'score': 0.6226047123369629, 'counts': [91267, 8922, 1786, 530], 'totals': [855246, 849295, 843344, 837393], 'precisions': [10.671432546892941, 1.0505183711195756, 0.211775977537043, 0.06329166830866749], 'bp': 1.0, 'sys_len': 855246, 'ref_len': 585268}


rouge:  {'rouge1': 0.09592971261464972, 'rouge2': 0.011937902530444775, 'rougeL': 0.07980817163201558, 'rougeLsum': 0.0745238647640932}


In [ ]:
eval('combined', c_test_df)

9229
9229
question:  [question] how can i determine if my new glasses are causing my headaches?
 [answer]
generated_answer:   I don't know. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know if my glasses are causing my headaches. I don't know